<a href="https://colab.research.google.com/github/Sreelakshmi2023/MAINPROJECT-GROUP12/blob/main/updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import dependencies

import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
!pip install keras-tcn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 12.6 MB/s eta 0:00:00


In [ ]:
from tcn import TCN

In [ ]:
!pip install chardet

In [ ]:
import chardet

In [ ]:
with open("datasetupdated.csv", "rb") as f:
    encoding = chardet.detect(f.read())["encoding"]

In [ ]:
data = pd.read_csv("datasetupdated.csv", encoding=encoding)

In [ ]:
with open("datasetupdated.csv", "rb") as f:
    for line in f:
        print(line.decode("utf-8", errors="ignore"))

In [ ]:
#load the dataset

df = pd.read_csv("datasetupdated.csv", encoding=encoding)

In [ ]:
df.head()

,url,type
0,br-icloud.com.br,malicious
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,malicious
4,http://adventure-nicaragua.net/index.php?optio...,malicious


In [ ]:
# Assuming your CSV has columns 'url' and 'type'
urls = df['url'].tolist()
labels = df['type'].tolist()

In [ ]:
# Convert labels to numerical format
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [ ]:
# Tokenization and padding
max_sequence_length = 100  # You can adjust this based on your data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(urls)
sequences = tokenizer.texts_to_sequences(urls)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, encoded_labels, test_size=0.2, random_state=42)

In [ ]:
y_test

[0 0 0 ... 0 0 0]


In [ ]:
# Model definition
model = Sequential()

In [ ]:
# Embedding layer
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=max_sequence_length))

In [ ]:
# CNN layer
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(2))


In [ ]:
# TCN layer
model.add(TCN(nb_filters=64, kernel_size=3, dilations=[1, 2, 4, 8, 16]))


In [ ]:
# LSTM layer
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for the reserved 0 index

from keras.layers import Embedding

vocab_size = len(tokenizer.word_index) + 1  # Add 1 for the reserved 0 index
embedding_dim = 64  # Adjust as needed

model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(64))


In [ ]:
# Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5


379/379 [==============================] - 15s 14ms/step - loss: 0.5854 - accuracy: 0.7300 - val_loss: 0.5708 - val_accuracy: 0.7428
Epoch 2/5
379/379 [==============================] - 3s 8ms/step - loss: 0.5835 - accuracy: 0.7308 - val_loss: 0.5708 - val_accuracy: 0.7428
Epoch 3/5
379/379 [==============================] - 4s 10ms/step - loss: 0.5835 - accuracy: 0.7308 - val_loss: 0.5712 - val_accuracy: 0.7428
Epoch 4/5
379/379 [==============================] - 3s 8ms/step - loss: 0.5840 - accuracy: 0.7308 - val_loss: 0.5748 - val_accuracy: 0.7428
Epoch 5/5
379/379 [==============================] - 3s 9ms/step - loss: 0.5839 - accuracy: 0.7308 - val_loss: 0.5706 - val_accuracy: 0.7428


In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)



95/95 [==============================] - 1s 6ms/step


In [ ]:
y_pred

array([[0.26550078],
       [0.26550078],
       [0.26550078],
       ...,
       [0.26550078],
       [0.26550078],
       [0.26550078]], dtype=float32)

In [ ]:
print(y_test.shape)
print(y_pred.shape)

(3033,)
(3033, 1)


In [ ]:
import numpy as np

In [ ]:
y_pred_labels = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_labels)

In [ ]:
accuracy

0.7428288822947576

In [ ]:
def predict_maliciousness(url):
    sequence = tokenizer.texts_to_sequences([url])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length)
    prediction = model.predict(padded_sequence)
    class_index = int(round(prediction[0][0]))
    classes = ['Benign', 'Malicious']
    return classes[class_index]

In [ ]:
user_url = "br-icloud.com.br"
prediction = predict_maliciousness(user_url)
print(f"The URL {user_url} is predicted as: {prediction}")

1/1 [==============================] - 0s 34ms/step
The URL br-icloud.com.br is predicted as: Benign
